# Выявление токсичных комментариев

### Содержание работы

**[Введение](#0)**

**[1. Загрузка и обработка данных](#1)** 
   
**[2. Обучение моделей](#2)**
     
**[3. Тестирование](#3)** 

**[Заключение](#4)**

## Описание проекта <a id="0"></a>

Интернет-магазин «Викишоп» запускает новый сервис. Пользователи смогут редактировать и дополнять описания товаров, как в вики-сообществах, то есть предлагать свои правки и комментировать изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Для идентификации токсичных комментариев, **необходимо построить модель, классифицирующую комментарии на позитивные и негативные**. В распоряжении имеется набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества F1 не меньше 0.75.

## 1. Загрузка и обработка данных <a id="1"></a>

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

import spacy

from tqdm import tqdm

Загрузим файл с данными и сохраним его в переменной `comments`.

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [5]:
comments = pd.read_csv('toxic_comments.csv')

In [6]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Все данные размечены. Посмотрим на примеры токсичных  и нетоксичных комментариев:

In [7]:
comments.sample(5)

,text,toxic
42045,"""\n\n Vandalism \n\nYou seem to have a questio...",0
136511,"""==Notability of Hero for now==\nA tag has bee...",0
100071,"Hi, the links work for me now, so must've been...",0
137068,Corrected the value on the number of signature...,0
123538,1. Active serive and ASU are commonly used ter...,0


In [8]:
comments[comments['toxic']==1].sample(3)['text']

57082    Cuz this admin is a maggot and slave to is POV...
58914    Glockers Deletion \n\nSuck my ballz fagget, do...
22449    Kathryn Bigelow is a patriot who makes great w...
Name: text, dtype: object

In [9]:
comments[comments['toxic']==0].sample(3)['text']

46816    All i can remember, in fact, is schnarquing yo...
25532          Preferably not. This section can be closed.
51125    Italian colonisation of the Americas \n\nHi Re...
Name: text, dtype: object

Посчитаем относительные размеры классов:

In [10]:
comments['toxic'].value_counts()/len(comments)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Почти 90% — нетоксичные комментарии, 10% — токсичные. При построении моделей необходимо учитывать дисбаланс классов.

Лемматизируем комментарии.

In [11]:
nlp = spacy.load('en')

comments['lemmatized'] = comments['text'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x)]))

Разобьём выборку на обучающую, валидационную и тестовую в отношении 3:1:1.

In [12]:
train_valid, test = train_test_split(comments, test_size=0.2, random_state=12345)
train, valid = train_test_split(train_valid, test_size=0.25, random_state=12345)

print(train.shape)
print(valid.shape)
print(test.shape)

(95742, 3)
(31914, 3)
(31915, 3)


In [14]:
corpus_train = train['lemmatized'].values.astype('U')
corpus_valid = valid['lemmatized'].values.astype('U')
corpus_test = test['lemmatized'].values.astype('U')

Создадим признаки для моделей — посчитаем TF-IDF для слов в комментариях. Зададим стартовые значения параметров min_df=0.05 и max_df=0.5.


In [15]:
count_tf_idf = TfidfVectorizer(token_pattern=r'[A-Za-z]{2,}', stop_words='english', min_df=.05, max_df=0.5)

count_tf_idf.fit(corpus_train)

X_train = count_tf_idf.transform(corpus_train)
X_valid = count_tf_idf.transform(corpus_valid)
X_test = count_tf_idf.transform(corpus_test)

In [16]:
y_train = train['toxic'].values
y_valid = valid['toxic'].values
y_test = test['toxic'].values

## 2. Обучение моделей <a id="2"></a>

Построим первую модель логистической регрессии.

In [17]:
lr = LogisticRegression(random_state=12345, class_weight='balanced')

lr.fit(X_train, y_train)

pred_train = lr.predict(X_train)
pred_valid = lr.predict(X_valid)

print('F1-score на обучающей выборке: ', f1_score(y_train, pred_train))
print('F1-score на валидационной выборке: ', f1_score(y_valid, pred_valid))

F1-score на обучающей выборке:  0.29142044430199715
F1-score на валидационной выборке:  0.29934484812388323


Значение F1-меры значительно меньше требуемого.

Построим модель дерева решений:

In [18]:
dt = DecisionTreeClassifier(random_state=12345, class_weight='balanced')

dt.fit(X_train, y_train)

pred_train = dt.predict(X_train)
pred_valid = dt.predict(X_valid)

print('F1-score на обучающей выборке: ', f1_score(y_train, pred_train))
print('F1-score на валидационной выборке: ', f1_score(y_valid, pred_valid))

F1-score на обучающей выборке:  0.4220538524303532
F1-score на валидационной выборке:  0.29110882251973913


F1-мера на валидационной выборке по-прежнему недостаточно велика.

Попробуем построить модель случайного леса:

In [19]:
rf = RandomForestClassifier(n_estimators=50, random_state=12345, class_weight='balanced')

rf.fit(X_train, y_train)

pred_train = rf.predict(X_train)
pred_valid = rf.predict(X_valid)

print('F1-score на обучающей выборке: ', f1_score(y_train, pred_train))
print('F1-score на валидационной выборке: ', f1_score(y_valid, pred_valid))

F1-score на обучающей выборке:  0.42613788243299067
F1-score на валидационной выборке:  0.29400030688967316


Результат по-прежнему неудовлетворительный.

Попробуем изменить параметры min_df и max_df:

In [20]:
param_grid = dict(min_df=[0, .01, .02, .03, .04, .05], max_df=[.2, .3, .4, .5])

In [22]:
results = []
for params in tqdm(ParameterGrid(param_grid)):

  count_tf_idf = TfidfVectorizer(token_pattern=r'[A-Za-z]{2,}', stop_words='english', min_df=params['min_df'], max_df=params['max_df'])
  count_tf_idf.fit(corpus_train)

  X_train = count_tf_idf.transform(corpus_train)
  X_valid = count_tf_idf.transform(corpus_valid)

  lr = LogisticRegression(random_state=12345, class_weight='balanced')
  lr.fit(X_train, y_train)

  results.append(dict(
    parameters=params,
    f1_train = f1_score(y_true=y_train, y_pred=lr.predict(X_train)),
    f1_valid = f1_score(y_true=y_valid, y_pred=lr.predict(X_valid)),
  ))

100%|██████████| 24/24 [06:57<00:00, 17.40s/it]


In [23]:
results = pd.DataFrame(results)

(results
 .sort_values('f1_valid', ascending=False)
 .style.bar(vmin=0, vmax=1)
)

,parameters,f1_train,f1_valid
0,"{'max_df': 0.2, 'min_df': 0}",0.842252,0.753302
6,"{'max_df': 0.3, 'min_df': 0}",0.841977,0.752436
18,"{'max_df': 0.5, 'min_df': 0}",0.841977,0.752436
12,"{'max_df': 0.4, 'min_df': 0}",0.841977,0.752436
19,"{'max_df': 0.5, 'min_df': 0.01}",0.409161,0.408681
7,"{'max_df': 0.3, 'min_df': 0.01}",0.409161,0.408681
13,"{'max_df': 0.4, 'min_df': 0.01}",0.409161,0.408681
1,"{'max_df': 0.2, 'min_df': 0.01}",0.406916,0.405057
14,"{'max_df': 0.4, 'min_df': 0.02}",0.347812,0.358142
8,"{'max_df': 0.3, 'min_df': 0.02}",0.347812,0.358142


При max_df=0.2 и min_df=0 значение F1-меры модели логистической регрессии незначительно превышает 0.75 на валидационной выборке.

Обновим признаки с учетом новых параметров:

In [24]:
count_tf_idf = TfidfVectorizer(token_pattern=r'[A-Za-z]{2,}', stop_words='english', min_df=0, max_df=0.2)

count_tf_idf.fit(corpus_train)

X_train = count_tf_idf.transform(corpus_train)
X_valid = count_tf_idf.transform(corpus_valid)
X_test = count_tf_idf.transform(corpus_test)

In [25]:
lr = LogisticRegression(random_state=12345, class_weight='balanced')

lr.fit(X_train, y_train)

pred_train = lr.predict(X_train)
pred_valid = lr.predict(X_valid)

print('F1-score на обучающей выборке: ', f1_score(y_train, pred_train))
print('F1-score на валидационной выборке: ', f1_score(y_valid, pred_valid))

F1-score на обучающей выборке:  0.8422522522522522
F1-score на валидационной выборке:  0.753302201467645


Попробуем ещё увеличить значение целевой метрики, изменив порог классификации.

In [27]:
prob_train = lr.predict_proba(X_train)
prob_valid = lr.predict_proba(X_valid)

prob_one_train = prob_train[:, 1]
prob_one_valid = prob_valid[:, 1]

In [40]:
results = []
for threshold in tqdm(np.arange(0.1, 0.9, 0.01)):
  
  pred_train = prob_one_train > threshold
  pred_valid = prob_one_valid > threshold

  results.append(dict(
    threshold=threshold,
    f1_train = f1_score(y_train, pred_train),
    f1_valid = f1_score(y_valid, pred_valid),
  ))

100%|██████████| 80/80 [00:04<00:00, 18.35it/s]


In [42]:
results = pd.DataFrame(results)

results.sort_values('f1_valid', ascending=False).head(20).style.bar(vmin=0, vmax=1)

,threshold,f1_train,f1_valid
61,0.710000,0.850960,0.784295
63,0.730000,0.846146,0.783862
62,0.720000,0.847864,0.783672
64,0.740000,0.842436,0.783618
60,0.700000,0.854189,0.783466
59,0.690000,0.858258,0.782474
65,0.750000,0.838021,0.782326
66,0.760000,0.834042,0.781286
58,0.680000,0.860756,0.780947
57,0.670000,0.863662,0.780570


Значение F1-меры на валидационной выборке превышает 0.78 при значении порога классификации 0.71.

Перейдём к модели случайного леса.

In [51]:
dt = DecisionTreeClassifier(random_state=12345, class_weight='balanced')

dt.fit(X_train, y_train)

pred_train = dt.predict(X_train)
pred_valid = dt.predict(X_valid)

print('F1-score на обучающей выборке: ', f1_score(y_train, pred_train))
print('F1-score на валидационной выборке: ', f1_score(y_valid, pred_valid))

F1-score на обучающей выборке:  0.9980377982030363
F1-score на валидационной выборке:  0.6790004141930139


F1 на валидационной выборке меньше требуемого значения 0.75, при этом модель явно перееобучена.

Узнаем глубину полученной модели:

In [46]:
dt.get_depth()

2049

Чтобы избежать переобученности модели, ограничим максимальную глубину дерева:

In [52]:
param_grid = dict(min_samples_split=[2], max_depth=[50, 100, 500, 1000, 1500])

In [55]:
results = []
for params in tqdm(ParameterGrid(param_grid)):
  dt = DecisionTreeClassifier(min_samples_split=params['min_samples_split'], max_depth=params['max_depth'], random_state=12345, class_weight='balanced')
  dt.fit(X_train, y_train)

  pred_train = dt.predict(X_train)
  pred_valid = dt.predict(X_valid)

  results.append(dict(
    parameters=params,
    f1_train = f1_score(y_true=y_train, y_pred=pred_train),
    f1_valid = f1_score(y_true=y_valid, y_pred=pred_valid),
  ))

100%|██████████| 5/5 [06:38<00:00, 79.61s/it]


In [56]:
results = pd.DataFrame(results)

results.sort_values('f1_valid', ascending=False).style.bar(vmin=0, vmax=1)

,parameters,f1_train,f1_valid
4,"{'max_depth': 1500, 'min_samples_split': 2}",0.998038,0.678414
1,"{'max_depth': 100, 'min_samples_split': 2}",0.906355,0.676488
0,"{'max_depth': 50, 'min_samples_split': 2}",0.818312,0.674147
3,"{'max_depth': 1000, 'min_samples_split': 2}",0.998038,0.673814
2,"{'max_depth': 500, 'min_samples_split': 2}",0.995909,0.672730


Значение метрики всё равно недостаточно велико.

Перейдём к модели случайного леса.

In [57]:
param_grid = dict(n_estimators=[100, 200, 300], max_depth=[50, 100, 500])

In [61]:
results = []
for params in tqdm(ParameterGrid(param_grid)):
  dt = RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'], random_state=12345, class_weight='balanced')
  dt.fit(X_train, y_train)
  results.append(dict(
    parameters=params,
    f1_train = f1_score(y_true=y_train, y_pred=dt.predict(X_train)),
    f1_valid = f1_score(y_true=y_valid, y_pred=dt.predict(X_valid)),
  ))

100%|██████████| 9/9 [1:11:27<00:00, 476.41s/it]


In [62]:
results = pd.DataFrame(results)

results.sort_values('f1_valid', ascending=False).style.bar(vmin=0, vmax=1)

,parameters,f1_train,f1_valid
6,"{'max_depth': 500, 'n_estimators': 100}",0.993822,0.671665
8,"{'max_depth': 500, 'n_estimators': 300}",0.993822,0.669877
7,"{'max_depth': 500, 'n_estimators': 200}",0.993928,0.669364
4,"{'max_depth': 100, 'n_estimators': 200}",0.725214,0.557194
3,"{'max_depth': 100, 'n_estimators': 100}",0.724256,0.556452
5,"{'max_depth': 100, 'n_estimators': 300}",0.723936,0.552990
0,"{'max_depth': 50, 'n_estimators': 100}",0.590877,0.502468
1,"{'max_depth': 50, 'n_estimators': 200}",0.593306,0.501303
2,"{'max_depth': 50, 'n_estimators': 300}",0.592093,0.498846


Значение метрики недостаточно велико.

Построим модель градиентного бустинга.

In [63]:
def lgb_f1_score(y_true, y_pred):

    return 'f1', f1_score(y_true, np.round(y_pred)), True

In [70]:
lgbm = LGBMClassifier(learning_rate=.1, n_estimators=500, class_weight='balanced')

lgbm.fit(X=X_train, y=y_train, eval_set=(X_valid, y_valid), eval_metric=lgb_f1_score, verbose=True, early_stopping_rounds=20)

[1]	valid_0's binary_logloss: 0.651451	valid_0's f1: 0.674304
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's binary_logloss: 0.616221	valid_0's f1: 0.657388
[3]	valid_0's binary_logloss: 0.585961	valid_0's f1: 0.682254
[4]	valid_0's binary_logloss: 0.559851	valid_0's f1: 0.671474
[5]	valid_0's binary_logloss: 0.537161	valid_0's f1: 0.692369
[6]	valid_0's binary_logloss: 0.517644	valid_0's f1: 0.672758
[7]	valid_0's binary_logloss: 0.499802	valid_0's f1: 0.675858
[8]	valid_0's binary_logloss: 0.48391	valid_0's f1: 0.678626
[9]	valid_0's binary_logloss: 0.469174	valid_0's f1: 0.7
[10]	valid_0's binary_logloss: 0.456308	valid_0's f1: 0.70109
[11]	valid_0's binary_logloss: 0.444908	valid_0's f1: 0.692168
[12]	valid_0's binary_logloss: 0.434458	valid_0's f1: 0.693378
[13]	valid_0's binary_logloss: 0.42471	valid_0's f1: 0.695089
[14]	valid_0's binary_logloss: 0.416151	valid_0's f1: 0.692935
[15]	valid_0's binary_logloss: 0.409073	valid_0's f1: 0.69489
[16]	valid_

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [96]:
f1_score(y_valid, lgbm.predict(X_valid))

0.7506804572672836

F1-мера на валидацонной выборке едва превышает пороговое значение 0.75. Попробуем изменить порог классификации для увеличения значения F1-меры.

In [72]:
prob_train = lgbm.predict_proba(X_train)
prob_valid = lgbm.predict_proba(X_valid)

prob_one_train = prob_train[:, 1]
prob_one_valid = prob_valid[:, 1]

In [73]:
results = []
for threshold in tqdm(np.arange(0.1, 0.9, 0.01)):
  
  pred_train = prob_one_train > threshold
  pred_valid = prob_one_valid > threshold

  results.append(dict(
    threshold=threshold,
    f1_train = f1_score(y_train, pred_train),
    f1_valid = f1_score(y_valid, pred_valid),
  ))

100%|██████████| 80/80 [00:04<00:00, 18.09it/s]


In [80]:
results = pd.DataFrame(results)

results.sort_values('f1_valid', ascending=False).head(20).style.bar(vmin=0, vmax=1)

,threshold,f1_train,f1_valid
60,0.700000,0.827759,0.779576
59,0.690000,0.828176,0.779550
63,0.730000,0.824858,0.779196
61,0.710000,0.826777,0.779163
62,0.720000,0.826310,0.779014
58,0.680000,0.827881,0.777985
64,0.740000,0.823851,0.777796
57,0.670000,0.827731,0.777761
66,0.760000,0.821338,0.776946
56,0.660000,0.827927,0.776892


При пороге 0.7 F1-score на валидационной выборке превышает 0.77.

Итого имеем 2 модели (логистическая регрессия и градиентный бустинг) с близкими значениями F1-меры, превышающими пороговое значение 0.75. Чтобы снизить variance предсказаний, попробуем объединить предсказания этих моделей. Найдём такие веса w1 и w2 (w1 + w2 = 1), чтобы сумма взвешенных предсказаний обеспечивала наибольшее значение метрики F1.

In [76]:
prob_one_train_lr = lr.predict_proba(X_train)[:, 1]
prob_one_valid_lr = lr.predict_proba(X_valid)[:, 1]

prob_one_train_lgbm = lgbm.predict_proba(X_train)[:, 1]
prob_one_valid_lgbm = lgbm.predict_proba(X_valid)[:, 1]

In [77]:
threshold = 0.7

pred_train_lr = prob_one_train_lr > threshold
pred_valid_lr = prob_one_valid_lr > threshold

pred_train_lgbm = prob_one_train_lgbm > threshold
pred_valid_lgbm = prob_one_valid_lgbm > threshold

In [92]:
results = []
for weight in tqdm(np.arange(0, 1.05, 0.05)):
  prob_one_train_ensemble = prob_one_train_lr * weight + prob_one_train_lgbm * (1 - weight)
  prob_one_valid_ensemble = prob_one_valid_lr * weight + prob_one_valid_lgbm * (1 - weight)

  threshold = 0.7

  pred_train_ensemble = prob_one_train_ensemble > threshold
  pred_valid_ensemble = prob_one_valid_ensemble > threshold

  results.append(dict(
    lr_weight=weight,
    f1_train = f1_score(y_train, pred_train_ensemble),
    f1_valid = f1_score(y_valid, pred_valid_ensemble),
  ))  

100%|██████████| 21/21 [00:01<00:00, 16.50it/s]


In [93]:
results = pd.DataFrame(results)

results.sort_values('f1_valid', ascending=False).style.bar(vmin=0, vmax=1)

,lr_weight,f1_train,f1_valid
10,0.500000,0.844430,0.791607
8,0.400000,0.841478,0.790779
9,0.450000,0.843426,0.790772
11,0.550000,0.845333,0.790084
7,0.350000,0.840343,0.789900
12,0.600000,0.846113,0.789474
13,0.650000,0.847117,0.789098
14,0.700000,0.848420,0.788666
15,0.750000,0.849949,0.788550
6,0.300000,0.838779,0.787639


Наилучший результат на валидационной выборке (0.79) показывает среднее арифмитическое предсказанных вероятностей двух моделей (w1 = w1 = 0.5).

## 3. Тестирование <a id="3"></a>

Получим предсказания на тестовой выборке:

In [95]:
weight_lr = 0.5
threshold = 0.7

prob_one_test_ensemble = lr.predict_proba(X_test)[:, 1] * weight_lr + lgbm.predict_proba(X_test)[:, 1] * (1 - weight_lr)

pred_test_ensemble = prob_one_test_ensemble > threshold

print('F1-score на тестовой выборке:', f1_score(y_true=y_test, y_pred=pred_test_ensemble))

F1-score на тестовой выборке: 0.7831620166421929


Значение F1-меры (0.78) больше требуемого уровня (0.75).

## Вывод <a id="4"></a>

Таким образом, были построены ряд моделей для выявления токсичных комментариев. Наилучшие результаты на валидационной выборке показали модели логистической регрессии (F1-score = 0.784) и градиентного бустинга (0.780). Усреднив вероятности, предсказанные этими 2 моделями, и использовав порог классификации 0.7, удалось увеличить значение F1-меры до 0.792. Значение целевой метрики на тестовой выборке составило 0.783, что больше требуемых 0.75.